In [284]:
import json, pathlib, random
from collections import defaultdict
import numpy as np
import pandas as pd


In [2]:
with open('lists.json') as f:
    j = json.load(f)

target_list = j['target']
guess_list = j["guess"]


In [292]:
def char_freq(lst):
    hist = defaultdict(int)
    for word in lst:
        for char in word:
            hist[char] += 1
    mx = max(hist.values())
    for char in hist:
        hist[char] /= mx
    return hist

def print_char_freq(cf):
    for char in sorted(list(cf.keys())):
        print(f'{char}: {cf[char]}')
        
def freq_score(word, cf):
    return sum(cf[x] for x in word) / len(word) 

def uniq_score(word):
    return (len(word) - len(set(word))) / (len(word) - 2)

In [288]:

cf = char_freq(target_list)
print_char_freq(cf)

a: 0.7939983779399837
b: 0.2278994322789943
c: 0.38686131386861317
d: 0.31873479318734793
e: 1.0
f: 0.18653690186536903
g: 0.2522303325223033
h: 0.31549067315490675
i: 0.5442011354420113
j: 0.021897810218978103
k: 0.170316301703163
l: 0.5831305758313058
m: 0.2562854825628548
n: 0.46634225466342255
o: 0.6115166261151662
p: 0.29764801297648014
q: 0.023519870235198703
r: 0.7291159772911597
s: 0.5425790754257908
t: 0.5912408759124088
u: 0.37875101378751014
v: 0.12408759124087591
w: 0.15815085158150852
x: 0.030008110300081103
y: 0.3446877534468775
z: 0.032441200324412


In [289]:
print(random.choice(guess_list))
print(random.choice(target_list))

swede
gipsy


In [331]:
dfg = pd.DataFrame([[w, freq_score(w, cf), uniq_score(w), 1] for w in guess_list], columns=['word', 'freq_score', 'uniq_score', 'is_guess_word'])
dft = pd.DataFrame([[w, freq_score(w, cf), uniq_score(w), 0] for w in target_list], columns=['word', 'freq_score', 'uniq_score', 'is_guess_word'])
df = dfg.append(dft)
df.set_index('word', inplace=True)

In [512]:
df
df.loc['hoppy']

freq_score       0.373398
uniq_score       0.333333
is_guess_word    1.000000
Name: hoppy, dtype: float64

In [362]:
class Env:
    def __init__(self, target_list, target_word=None):
        if target_word:
            self.target = target_word
        else:
            self.target = random.choice(target_list)
        
    def submit_guess(self, guess):
        wrongplace = [0] * len(self.target)
        hints = np.zeros(len(self.target))
        rightplace = [guess[n] == chrt for n,chrt in enumerate(self.target)]
        
        for n,chrt in enumerate(self.target):
            if rightplace[n] == 1: continue #this character has already been scored, skip it
            for m,chrg in enumerate(guess):
                if n == m: continue # we've already checked rightplace matches above
                if chrt != chrg: continue
                if wrongplace[m] == 1: continue
                if rightplace[m] == 1: continue
                
                wrongplace[m] = 1
                break

        for i in range(len(self.target)):
            hints[i] = 2 if rightplace[i] == 1 else wrongplace[i]
        
        return hints
    
def hint_to_hinty(hint):
    #hint takes form [0,1,2,1,0]
    #hinty takes form {2:[2], 1:[1,3], 0:[0,4]}
    hinty = {}
    for n in [0,1,2]:
        hinty[n] = [i for i, x in enumerate(hint) if x == n]
    #print(f'hint_to_hinty() {hint}, {hinty}')
    return hinty
    
def validate_against_hint(word, guess, hint):
    return validate_against_hinty(word, guess, hint_to_hinty(hint))

def validate_against_hinty(word, guess, hinty):
    #hinty takes form {2:[idx,..], 1:[idx,..], 0:[idx,..]}
    for idx in hinty[2]: # check the fixed letters first
        if word[idx] != guess[idx]:
            return False
    for idx in hinty[0]:
        #get the number of times char appears in target word (minus the times it appears in the correct location)
        indices = [i for i,x in enumerate(word) if x == guess[idx] and i not in hinty[2]]
        #get number of times char appears in guess word in the wrong location
        indices_g = [n for n,x in enumerate(guess) if x == guess[idx] and n in hinty[1]]
        #we already know that there is one not-exist hint for this char, so
        #if there are more fewer wrong location hints for this letter than there are actual occurrences of the letter
        #then the hint does not validate against this word
        if len(indices) > len(indices_g):
            return False
    for idx in hinty[1]:
        if word[idx] == guess[idx]:
            return False
        #get all the indices of the character in the target word
        indices = [i for i,x in enumerate(word) if x == guess[idx] and i not in hinty[2]]
        #remove all the indices where there is already a fixed position hint
        
        #now count all the occurences of the char in guess where the location is wrong
        indices_g = [i for i,x in enumerate(guess) if x == guess[idx] and i in hinty[1]]
        #if there are more wrong loc hints for this char than there are actual occurrences, then it must be bogus
        if len(indices) < len(indices_g):
            return False
    return True            
    

In [317]:
e_simple = Env(target_list, target_word='abcde')
tests_simple = {'abcde': [2,2,2,2,2],
         'acbde': [2,1,1,2,2],
         'azcde': [2,0,2,2,2],
         'aacde': [2,0,2,2,2],
         'zacde': [0,1,2,2,2],
         'zzdzz': [0,0,1,0,0],
         'zzddz': [0,0,0,2,0],
         'zdddz': [0,0,0,2,0],
         'ddddd': [0,0,0,2,0],
         'zzzdd': [0,0,0,2,0],
         'zzdez': [0,0,1,1,0]}

e_repeat = Env(target_list, target_word='abcae')
tests_repeat = {'abcde': [2,2,2,0,2],
         'acbde': [2,1,1,0,2],
         'azcde': [2,0,2,0,2],
         'aacde': [2,1,2,0,2],
         'zacde': [0,1,2,0,2],
         'zzdzz': [0,0,0,0,0],
         'zzddz': [0,0,0,0,0],
         'zdddz': [0,0,0,0,0],
         'ddddd': [0,0,0,0,0],
         'zzzdd': [0,0,0,0,0],
         'zzdez': [0,0,0,1,0],
         'aaaaa': [2,0,0,2,0],
         'aaaza': [2,1,0,0,0],
         'zaazz': [0,1,1,0,0],
         'zaaza': [0,1,1,0,0]}

for e,tests in [(e_simple, tests_simple),(e_repeat, tests_repeat)]:
    for guess,expected in tests.items():
        #guess = random.choice(guess_list + target_list)
        actual = e.submit_guess(guess)
        hinty = hint_to_hinty(expected)
        hinty_valid = validate_against_hinty(e.target, guess, hinty)
        print(e.target, guess, actual, expected, expected == actual, hinty_valid)

abcde abcde [2. 2. 2. 2. 2.] [2, 2, 2, 2, 2] [ True  True  True  True  True] True
abcde acbde [2. 1. 1. 2. 2.] [2, 1, 1, 2, 2] [ True  True  True  True  True] True
abcde azcde [2. 0. 2. 2. 2.] [2, 0, 2, 2, 2] [ True  True  True  True  True] True
abcde aacde [2. 0. 2. 2. 2.] [2, 0, 2, 2, 2] [ True  True  True  True  True] True
abcde zacde [0. 1. 2. 2. 2.] [0, 1, 2, 2, 2] [ True  True  True  True  True] True
abcde zzdzz [0. 0. 1. 0. 0.] [0, 0, 1, 0, 0] [ True  True  True  True  True] True
abcde zzddz [0. 0. 0. 2. 0.] [0, 0, 0, 2, 0] [ True  True  True  True  True] True
abcde zdddz [0. 0. 0. 2. 0.] [0, 0, 0, 2, 0] [ True  True  True  True  True] True
abcde ddddd [0. 0. 0. 2. 0.] [0, 0, 0, 2, 0] [ True  True  True  True  True] True
abcde zzzdd [0. 0. 0. 2. 0.] [0, 0, 0, 2, 0] [ True  True  True  True  True] True
abcde zzdez [0. 0. 1. 1. 0.] [0, 0, 1, 1, 0] [ True  True  True  True  True] True
abcae abcde [2. 2. 2. 0. 2.] [2, 2, 2, 0, 2] [ True  True  True  True  True] True
abcae acbde [2. 

In [112]:
def random_guess(guess_list, target_list):
    guess_idx = random.randint(0, len(guess_list) + len(target_list))
    is_guess = guess_idx < len(guess_list)
    if is_guess:
        word = guess_list[guess_idx]
    else:
        word = target_list[guess_idx - len(guess_list)]
    return word, is_guess

In [504]:
def history_to_state(guesses, history):
    #print(history)
    #so the state is going to be:
        #  The number of green locations we know
        #  The number of other letters we know to be in the word
        #  The sequence number of the guess (1st guess, 2nd guess etc.)
    
    #the number of locations which were green at some point in the history
    num_green_locs = np.count_nonzero(history.max(axis=0) == 2)
    
    green_chars = [guesses[x][y] for x,y in np.argwhere(history == 2) ]
    orange_chars = [guesses[x][y] for x,y in np.argwhere(history == 1) ]
    num_other_letters = len(set(orange_chars) - set(green_chars))
    
    sequence_number = history.shape[0]
    
    return np.array([num_green_locs, num_other_letters, sequence_number]) / 5

def word_to_action(word, guesses, history):
    return dfword_to_action((word, df.loc[word]), guesses, history)
    
def dfword_to_action(dfword, guesses, history):
    #the action is going to be a word that we will submit next
    #for the purposes of feeding into the model, we will represent the action word as:
    #  how many of the entries in the hint history this word conforms to
    #  how many untried letters it gives us
    #  the number of uniq letters in the word
    #  the frequency of the letters in the word
    #  whether or not the word is in the guess list (as opposed to the target list)
    word = dfword[0]
    dfword = dfword[1]
    if guesses:
        conforms_to_history = sum([int(validate_against_hint(word,g,history[i])) for i,g in enumerate(guesses)]) / len(guesses)
    else: # we haven't made any guess yet, so this must conform
        conforms_to_history = 1.0
    num_untried_letters = len(set(word) - set(''.join(guesses))) / 5 #normalise to 1
    return np.array([conforms_to_history, num_untried_letters, dfword['freq_score'], dfword['uniq_score'], dfword['is_guess_word']])
    

In [517]:
#'beast'
#Env(target_list, target_word='beast').submit_guess('treat')
actual = history_to_state(['treat'], np.array([[0.0, 0.0, 1.0, 1.0, 2.0]]))
expected = [0.2, 0.4, 0.2]
print(expected, actual, expected == actual)

actual = word_to_action('feast', ['treat'], np.array([[0.0, 0.0, 1.0, 1.0, 2.0]]))
expected = [1.0, 0.4, 0.62287105, 0.0, 0.0]
print(expected, actual, expected == actual)


[0.2, 0.4, 0.2] [0.2 0.4 0.2] [ True  True  True]
[1.0, 0.4, 0.62287105, 0.0, 0.0] [1.         0.4        0.62287105 0.         0.        ] [ True  True False  True  True]


In [491]:
def construct_actions(guesses, history):
    return np.array([dfword_to_action(dfword, guesses, history) for dfword in df.iterrows()])

In [505]:
num_guesses = 6
e = Env(target_list)

print(e.target)
num_letters = len(e.target)
history = np.array([[]])
guesses = []
rewards = []
for i in range(num_guesses):
    #guess, is_guess_list = random_guess(guess_list, target_list)
    actions = construct_actions(guesses, history)
    state = history_to_state(guesses, history)
    #here feed it into a model to choose the word
    #guess, value = np.argmax(model(state)) # but do this epsilon greedy
    
    #print(actions)
    hints = e.submit_guess(guess)
    
    print(f'======={guess}========')
    print(list(zip(guesses,history)))
    if history.size == 0:
        history = np.expand_dims(hints,0)
    else:
        history = np.row_stack([history, hints])
    guesses.append(guess)
    if hints.sum() == num_letters * 2 or i == num_guesses - 1:
        reward = hints.sum()
        done = True
    else:
        reward = -1
        done = False
    
    

    
#so the state is going to be:
#  The number of green locations we know
#  The number of other letters we know to be in the word
#  The sequence number of the guess (1st guess, 2nd guess etc.)

#the action is going to be a word that we will submit next
#for the purposes of feeding into the model, we will represent the action word as:
#  whether or not it conforms to the hint history
#  how many new letters it gives us
#  the number of uniq letters in the word
#  the frequency of the letters in the word

#the reward is going to be:
#  -1 on all states except the last one
#  on the last state (which can either be after guess 6 or on guessing the correct word):
#    the sum of the last hint (ie. 2 for a correct letter/position combo, 1 for a letter in the wrong place)

badge
[[1.         0.8        0.64444444 0.33333333 1.        ]
 [1.         0.6        0.65190592 0.66666667 1.        ]
 [1.         0.8        0.57696675 0.33333333 1.        ]
 ...
 [1.         1.         0.60032441 0.         0.        ]
 [1.         0.8        0.64282238 0.33333333 0.        ]
 [1.         1.         0.55523114 0.         0.        ]]
=======nenes========
[]
[0.  0.  0.2]
[1.         0.6        0.69505272 0.66666667 1.        ]
[[1.         0.6        0.64444444 0.33333333 1.        ]
 [0.         0.6        0.65190592 0.66666667 1.        ]
 [0.         0.8        0.57696675 0.33333333 1.        ]
 ...
 [0.         0.8        0.60032441 0.         0.        ]
 [0.         0.8        0.64282238 0.33333333 0.        ]
 [0.         0.6        0.55523114 0.         0.        ]]
=======crags========
[('nenes', array([0., 1., 0., 0., 0.]))]
[0.  0.2 0.2]
[0.         0.8        0.54095702 0.         1.        ]
[[0.5        0.4        0.64444444 0.33333333 1.        ]
